# Importing libraries

In [1]:
# Importing Libraries
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

from keras.models import load_model
from matplotlib import pyplot


# Importing datasets

In [2]:

# Path to local data directory
data_dir = '/Users/air/Desktop/FinalProject/Data'

# Load data

X_labeled = np.load(f'{data_dir}/X_labeled.npy')
y_labeled = np.load(f'{data_dir}/y_labeled.npy')

X_val = np.load(f'{data_dir}/X_val.npy')
y_val = np.load(f'{data_dir}/y_val.npy')


## Importing Model

In [3]:
from tensorflow.keras.models import load_model

model_path = "/Users/air/Desktop/FinalProject/Models/Rot-PretextTask15epochs.h5"  # specify your path here
model = load_model(model_path)



In [4]:
# Checking the changes in Model Architecture
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 64)        9472      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 16, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 16, 128)       73856     
                                                                 
 conv2d_2 (Conv2D)           (None, 16, 16, 128)       147584    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 8, 8, 128)        0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 8, 8, 256)         2

# Model

In [5]:
# Removing the top layer and addding a new top layer
model.pop()
model.add(keras.layers.Dense(10, name='dense_3', activation='softmax'))

In [6]:
# Checking the changes in Model Architecture
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 64)        9472      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 16, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 16, 128)       73856     
                                                                 
 conv2d_2 (Conv2D)           (None, 16, 16, 128)       147584    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 8, 8, 128)        0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 8, 8, 256)         2

## Modifying Model

In [7]:
# See layer.trainbale boolean of each layer
for layer in model.layers:
    print(layer.name, layer.trainable)

conv2d True
max_pooling2d True
conv2d_1 True
conv2d_2 True
max_pooling2d_1 True
conv2d_3 True
conv2d_4 True
max_pooling2d_2 True
flatten True
dense True
dropout True
dense_1 True
dropout_1 True
dense_3 True


In [8]:
# Freezing the Convolutional Layers while keeping Dense layers as Trainable
for layer in model.layers:
    if layer.name in ['dense_3', 'dense_1', 'dense', 'dropout', 'dropout_1']:
      layer.trainable=True
    else:
      layer.trainable=False

In [9]:
# Checking if the changes in 'Trainable' status of each layer have taken place
for layer in model.layers:
    print(layer.name, layer.trainable)

conv2d False
max_pooling2d False
conv2d_1 False
conv2d_2 False
max_pooling2d_1 False
conv2d_3 False
conv2d_4 False
max_pooling2d_2 False
flatten False
dense True
dropout True
dense_1 True
dropout_1 True
dense_3 True


# Bayesian Optimisation

In [11]:
%%time
from bayes_opt import BayesianOptimization
from tensorflow.keras.callbacks import EarlyStopping

def BayesOptimization(learning_rate, batch_size):
    # Convert hyperparameters to appropriate format
    batch_size = int(batch_size)

    # Compile model
    model.compile(loss="sparse_categorical_crossentropy", optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=learning_rate), metrics=["accuracy"])
    
    # Early stopping
    # disabled because it prevents convergence
    #early_stopping = EarlyStopping(monitor='val_loss', patience=4) 

    # Train your model and return the validation error
    history = model.fit(X_labeled, y_labeled, batch_size=batch_size, validation_data=(X_val, y_val), epochs=10)
    # Using the negative validation accuracy as a score to maximize accuracy
    score = -history.history['val_accuracy'][-1]

    return score

def optimize_cnn():
    optimizer = BayesianOptimization(
        f=BayesOptimization,
        pbounds={'learning_rate': (0.0001, 0.1), 'batch_size': (32, 512)},
        verbose=2,
        random_state=1,
    )
    # n_iter value to change the number of attemps
    optimizer.maximize(init_points=10, n_iter=30)

    # Print Optimizer results
    print(optimizer.max)
    
    # Return optimizer for metrics
    return optimizer

optimizer = optimize_cnn()

|   iter    |  target   | batch_... | learni... |
-------------------------------------------------
Epoch 1/10


2023-08-03 14:33:29.001965: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


44/44 [==============================] - 11s 243ms/step - loss: 3.5651 - accuracy: 0.1045 - val_loss: 2.3021 - val_accuracy: 0.0994
Epoch 2/10
44/44 [==============================] - 11s 259ms/step - loss: 2.3553 - accuracy: 0.1028 - val_loss: 2.2914 - val_accuracy: 0.1098
Epoch 3/10
44/44 [==============================] - 11s 262ms/step - loss: 2.3039 - accuracy: 0.1107 - val_loss: 2.1804 - val_accuracy: 0.1680
Epoch 4/10
44/44 [==============================] - 12s 265ms/step - loss: 2.2892 - accuracy: 0.1158 - val_loss: 2.1800 - val_accuracy: 0.1770
Epoch 5/10
44/44 [==============================] - 12s 266ms/step - loss: 2.2657 - accuracy: 0.1150 - val_loss: 2.2239 - val_accuracy: 0.1684
Epoch 6/10
44/44 [==============================] - 12s 266ms/step - loss: 2.2803 - accuracy: 0.1137 - val_loss: 2.2781 - val_accuracy: 0.1212
Epoch 7/10
44/44 [==============================] - 12s 267ms/step - loss: 2.3076 - accuracy: 0.1050 - val_loss: 2.3044 - val_accuracy: 0.1008
Epoch 8/10

## Saving Model

In [ ]:
# Save the model
model.save('/Users/air/Desktop/FinalProject/Models/TargetBayesOpti.h5')


In [ ]:
# View the Target Model Architecture
model.summary()

# Metrics


In [ ]:
# Evaluate Model
eval_result = model.evaluate(X_val, y_val)
print("\nTest loss, Test accuracy:", eval_result)

# Plots
fig, axs = plt.subplots(2)

# summarize history for accuracy
axs[0].plot(optimizer.max['params']['history'].history['accuracy'])
axs[0].plot(optimizer.max['params']['history'].history['val_accuracy'])
axs[0].set_title('Model accuracy')
axs[0].set_ylabel('Accuracy')
axs[0].set_xlabel('Epoch')
axs[0].legend(['train', 'validation'], loc='upper left')

# summarize history for loss
axs[1].plot(optimizer.max['params']['history'].history['loss'])
axs[1].plot(optimizer.max['params']['history'].history['val_loss'])
axs[1].set_title('Model loss')
axs[1].set_ylabel('Loss')
axs[1].set_xlabel('Epoch')
axs[1].legend(['train', 'validation'], loc='upper left')

# Plotting the Metrics
plt.tight_layout()
plt.show()me 'optimizer' is not defined
